## Load the Dataset

In [63]:
# importing libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set a style for the plots
sns.set(style='whitegrid')

# Suppressing warnings
import warnings
warnings.filterwarnings('ignore')


In [4]:
# import file 

df = pd.read_csv('../data/ecom_orders.csv')

# Display first five rows of the DataFrame

print(df.head())

                               order_id  product_id product_name   category  \
0  8b1424e0-1f73-4229-91fa-299bde1e7941        8946         open      Books   
1  02374a38-64bf-4034-967e-6cbfaed3f7fd        3252     standard       Toys   
2  77745bf4-ed01-4e75-999a-f25f57cd8816        6948      discuss  Furniture   
3  2537c981-eae8-452b-bd0a-908fc37b82b4        6717         yeah   Clothing   
4  27db871d-58f5-49d1-a7e3-3533998f0b5b        2266         they      Books   

    price  quantity  order_date  customer_id marketplace order_status  \
0  236.75         5  2023-09-30         5266       Olist    Delivered   
1  252.04         2  2022-11-23         9332        eBay      Pending   
2  480.05         3  2024-09-20         8264        eBay      Pending   
3  122.04         1  2022-03-15         2538      Amazon      Pending   
4  434.80         6  2023-07-01         5412        eBay      Pending   

  payment_method  shipment_cost           city           state  
0         PayPal     

## Clean and Preprocess the Data

In [6]:
# Check for missing values
df.isnull().sum()

# Fill or drop missing values as needed
df = df.dropna()



# Convert 'order_date' to datetime and ensure 'quantity' is numeric
df['order_date'] = pd.to_datetime(df['order_date'])
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')

# Create 'year_month' column for monthly grouping
df['year_month'] = df['order_date'].dt.to_period('M')

## Perform the Required Analysis

### Sales from Top 20 Products

In [59]:
# Group by product_id and sum the quantities
top_20_products = df.groupby('product_id')['quantity'].sum().nlargest(20)

# Merge with product name and category for more information
top_20_products_info = pd.merge(top_20_products.reset_index(), df[['product_id', 'product_name', 'category']].drop_duplicates(), on='product_id')

# View the result
top_20_products_info.head()


,product_id,quantity,product_name,category
0,8242,150,fill,Clothing
1,8242,150,professor,Electronics
2,8242,150,experience,Furniture
3,8242,150,skill,Electronics
4,8242,150,investment,Clothing


### Top 20 Products in Each Month

In [50]:
# Convert order_date to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Extract year and month
df['year_month'] = df['order_date'].dt.to_period('M')

# Group by year_month and product_id, and calculate total sales quantity
monthly_top_20 = df.groupby(['year_month', 'product_id'])['quantity'].sum().reset_index()

# Sort by each month and get the top 20 products
monthly_top_20 = monthly_top_20.groupby('year_month').apply(lambda x: x.nlargest(20, 'quantity')).reset_index(drop=True)

# Merge with product name and category for more information
monthly_top_20_info = pd.merge(monthly_top_20, df[['product_id', 'product_name', 'category']].drop_duplicates(), on='product_id')

# View the result
monthly_top_20_info.head()


,year_month,product_id,quantity,product_name,category
0,2021-10,9556,31,thing,Toys
1,2021-10,9556,31,year,Books
2,2021-10,9556,31,throughout,Books
3,2021-10,9556,31,enter,Electronics
4,2021-10,9556,31,city,Clothing


### Top 1%, 5% Product Sales in Each Month

#### Calculate Percentile Values

In [40]:
# Function to calculate top N percent of products
def top_percent(df, percent):
    top_n = int(len(df) * percent / 100)
    return df.nlargest(top_n, 'quantity')

# Calculate top 1% and 5% for each month
top_1_percent = df.groupby('year_month').apply(lambda x: top_percent(x.groupby('product_id')['quantity'].sum().reset_index(), 1))
top_5_percent = df.groupby('year_month').apply(lambda x: top_percent(x.groupby('product_id')['quantity'].sum().reset_index(), 5))

# View the results
top_1_percent.head()
top_5_percent.head()


product_id  quantity
year_month                           
2021-10    1340        9556        31
           107         1804        20
           539         4310        20
           917         6840        19
           929         6903        19

### Top Product Sales in Each Category

In [42]:
# Group by category and product_id, and sum the quantities
category_top_products = df.groupby(['category', 'product_id'])['quantity'].sum().reset_index()

# Sort by each category and get the top product
category_top_product = category_top_products.groupby('category').apply(lambda x: x.nlargest(1, 'quantity')).reset_index(drop=True)

# Merge with product name for more information
category_top_product_info = pd.merge(category_top_product, df[['product_id', 'product_name']].drop_duplicates(), on='product_id')

# View the result
category_top_product_info.head()


,category,product_id,quantity,product_name
0,Beauty,6783,56,my
1,Beauty,6783,56,adult
2,Beauty,6783,56,into
3,Beauty,6783,56,seat
4,Beauty,6783,56,age
